# Lab 9.5: Relu + Xavier MNIST Classifier

Edited By Steve Ive

Reference from

https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-09_4_mnist_nn_deep.ipynb

## Imports

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random

In [44]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(1)
torch.manual_seed(1)

if device == 'cuda':
    torch.cuda.manual_seed_all(1)

## Load MNIST Data

In [45]:
mnist_train = datasets.MNIST(root = 'MNIST_data/',
                             download=True,
                             transform=transforms.ToTensor(),
                             train=True)
mnist_test = datasets.MNIST(root = 'MNIST_data/',
                            download=True,
                            transform=transforms.ToTensor(),
                            train= False)

## Set Hyperparameters

In [46]:
learning_rate = 0.001
batch_size = 100
training_epochs = 15

In [47]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, shuffle = True, drop_last=True, batch_size = batch_size)

## Define Model

In [48]:
class ReLuXavierMNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.sq = nn.Sequential(
            nn.Linear(784, 516),
            nn.ReLU(),
            nn.Linear(516, 516),
            nn.ReLU(),
            nn.Linear(516, 516),
            nn.ReLU(),
            nn.Linear(516, 516),
            nn.ReLU(),
            nn.Linear(516, 10)
        )
        self.weightInitializer()

    def weightInitializer(self):
        for index, layer in enumerate(self.sq):
            if index != 1 and index != 3 and index != 5 and index != 7:
                nn.init.xavier_uniform_(layer.weight)

    def forward(self, x):
        return self.sq(x)

In [49]:
model = ReLuXavierMNISTClassifier().to(device)

In [50]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Train Model

In [51]:
total_batch = len(data_loader)

for epoch in range(training_epochs):

    avg_cost = 0

    for X, Y in data_loader:

        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        #prediction
        pred = model(X)

        #cost
        cost = F.cross_entropy(pred, Y).to(device)

        #Reduce cost
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost

    avg_cost = avg_cost / total_batch

    print('Epoch: {} / 15, Cost: {:.6f}'.format(epoch+1, avg_cost.item()))

print('Learning Finished')

Epoch: 1 / 15, Cost: 0.212796
Epoch: 2 / 15, Cost: 0.087777
Epoch: 3 / 15, Cost: 0.061607
Epoch: 4 / 15, Cost: 0.048517
Epoch: 5 / 15, Cost: 0.043476
Epoch: 6 / 15, Cost: 0.031130
Epoch: 7 / 15, Cost: 0.030316
Epoch: 8 / 15, Cost: 0.023586
Epoch: 9 / 15, Cost: 0.023560
Epoch: 10 / 15, Cost: 0.021674
Epoch: 11 / 15, Cost: 0.021906
Epoch: 12 / 15, Cost: 0.017494
Epoch: 13 / 15, Cost: 0.015295
Epoch: 14 / 15, Cost: 0.019556
Epoch: 15 / 15, Cost: 0.014361


In [52]:
#Accuracy Test and Check Prediction

with torch.no_grad():

    X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    pred = model(X_test)
    correct_prediction = (torch.argmax(pred, 1) == Y_test)
    accuracy = correct_prediction.float().mean()

    print('Accuracy: {}'.format(accuracy))

    r = random.randint(0, len(mnist_test) - 1)

    X_single_prediction = X_test[r]
    Y_single_prediction = Y_test[r]

    print('Label: {}'.format(Y_single_prediction.item()))
    print('Prediction: {}'.format(torch.argmax(model(X_single_prediction)).item()))

Accuracy: 0.9817000031471252
Label: 4
Prediction: 4
